In [1]:
import plotly.express as px
from collections import defaultdict
from multiprocessing import Pool
import multiprocessing
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import postprocess_general
import postprocess_randomness
import postprocess_randomness_enhanced
import postprocess_initpos
import feature
import matplotlib.pyplot as plt
import statistics
def default_value():
    return 0
def default_list():
    return []
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams["figure.figsize"] = (10,5)

In [2]:
path = "../data/"
dst = "../processed_initpos/"
dir_list = os.listdir(dst)

winner_pos=[[],[],[],[]]
winner_fan=[[],[],[],[]]
winner_pos_adj=[[],[],[],[]]
winner_fan_adj=[[],[],[],[]]

count=0
alt_list=defaultdict(default_value)
    
pbar = tqdm(dir_list)
for file in pbar:
    count+=1
    (winner_id,
    _winner_fan,
    completing_ids,
    completing_id_fans) = postprocess_initpos.load_from_result(dst, file)
    for i in range(4):
        losing_pos_adj=False
        if winner_id==i:
            winner_pos[i].append(1)
            winner_pos_adj[i].append(1)
            winner_fan[i].append(_winner_fan)
            winner_fan_adj[i].append(_winner_fan)
        else:
            winner_pos[i].append(0)
            winner_fan[i].append(0)
            for j in range(len(completing_ids)):
                if completing_ids[j]==i:
                    losing_pos_adj=True
                    winner_pos_adj[i].append(1)
                    winner_fan_adj[i].append(completing_id_fans[j])
            if not losing_pos_adj:
                winner_pos_adj[i].append(0)
                winner_fan_adj[i].append(0)
print("Processing Done!")
                    
        

100%|██████████| 98209/98209 [01:59<00:00, 819.54it/s]

Processing Done!


In [17]:
# prefix sum scan
total_length = 45001
cum_winner_pos=[[],[],[],[]]
cum_winner_fan=[[],[],[],[]]
cum_winner_pos_adj=[[],[],[],[]]
cum_winner_fan_adj=[[],[],[],[]]
for i in range(total_length):
    # file_id = np.random.randint(0, len(dir_list))
    file_id=i
    for index in range(4):
        if i==0:
            prev_winner_pos=0
            prev_winner_fan=0
            prev_winner_pos_adj=0
            prev_winner_fan_adj=0
        else:
            prev_winner_pos=cum_winner_pos[index][-1]
            prev_winner_fan=cum_winner_fan[index][-1]
            prev_winner_pos_adj=cum_winner_pos_adj[index][-1]
            prev_winner_fan_adj=cum_winner_fan_adj[index][-1]
        cum_winner_pos[index].append(prev_winner_pos+winner_pos[index][file_id])
        cum_winner_fan[index].append(prev_winner_fan+winner_fan[index][file_id])
        cum_winner_pos_adj[index].append(prev_winner_pos_adj+winner_pos_adj[index][file_id])
        cum_winner_fan_adj[index].append(prev_winner_fan_adj+winner_fan_adj[index][file_id])

In [18]:
peek_index=45000
cum = 0
for i in range(4):
    cum+=cum_winner_pos_adj[i][peek_index]
for i in range(4):
    print("Pos {}'s adjusted win rate is {:.3f}".format(i,cum_winner_pos_adj[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_pos_adj[0][peek_index]-cum_winner_pos_adj[3][peek_index])/cum))

Pos 0's adjusted win rate is 0.261
Pos 1's adjusted win rate is 0.257
Pos 2's adjusted win rate is 0.248
Pos 3's adjusted win rate is 0.234
Maximum Diff is 0.026457161057437097


In [19]:
cum=0
for i in range(4):
    cum+=cum_winner_pos[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_pos[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_pos[0][peek_index]-cum_winner_pos[3][peek_index])/cum))

Pos 0's raw win rate is 0.267
Pos 1's raw win rate is 0.258
Pos 2's raw win rate is 0.246
Pos 3's raw win rate is 0.228
Maximum Diff is 0.039665875479276975


In [20]:
cum=0
for i in range(4):
    cum+=cum_winner_fan[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_fan[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_fan[0][peek_index]-cum_winner_fan[3][peek_index])/cum))

Pos 0's raw win rate is 0.271
Pos 1's raw win rate is 0.256
Pos 2's raw win rate is 0.247
Pos 3's raw win rate is 0.227
Maximum Diff is 0.044259101211629404


In [21]:
cum=0
for i in range(4):
    cum+=cum_winner_fan_adj[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_fan_adj[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_fan_adj[0][peek_index]-cum_winner_fan_adj[3][peek_index])/cum))

Pos 0's raw win rate is 0.264
Pos 1's raw win rate is 0.254
Pos 2's raw win rate is 0.248
Pos 3's raw win rate is 0.234
Maximum Diff is 0.030369683964104565


### Conclusion: By Adding Completing Round Mechanism, initial position advantage can be decreased by ~33%.

In [23]:
path = "../data/"
dst = "../processed_initpos_alt/"
dir_list = os.listdir(dst)

winner_pos=[[],[],[],[]]
winner_fan=[[],[],[],[]]
winner_pos_adj=[[],[],[],[]]
winner_fan_adj=[[],[],[],[]]

count=0
alt_list=defaultdict(default_value)
    
pbar = tqdm(dir_list)
for file in pbar:
    count+=1
    (winner_id,
    _winner_fan,
    completing_ids,
    completing_id_fans) = postprocess_initpos.load_from_result(dst, file)
    for i in range(4):
        losing_pos_adj=False
        if winner_id==i:
            winner_pos[i].append(1)
            winner_pos_adj[i].append(1)
            winner_fan[i].append(_winner_fan)
            winner_fan_adj[i].append(_winner_fan)
        else:
            winner_pos[i].append(0)
            winner_fan[i].append(0)
            for j in range(len(completing_ids)):
                if completing_ids[j]==i:
                    losing_pos_adj=True
                    winner_pos_adj[i].append(1)
                    winner_fan_adj[i].append(completing_id_fans[j])
            if not losing_pos_adj:
                winner_pos_adj[i].append(0)
                winner_fan_adj[i].append(0)
print("Processing Done!")

100%|██████████| 98209/98209 [01:38<00:00, 993.52it/s] 

Processing Done!


In [24]:
# prefix sum scan
total_length = 45001
cum_winner_pos=[[],[],[],[]]
cum_winner_fan=[[],[],[],[]]
cum_winner_pos_adj=[[],[],[],[]]
cum_winner_fan_adj=[[],[],[],[]]
for i in range(total_length):
    # file_id = np.random.randint(0, len(dir_list))
    file_id=i
    for index in range(4):
        if i==0:
            prev_winner_pos=0
            prev_winner_fan=0
            prev_winner_pos_adj=0
            prev_winner_fan_adj=0
        else:
            prev_winner_pos=cum_winner_pos[index][-1]
            prev_winner_fan=cum_winner_fan[index][-1]
            prev_winner_pos_adj=cum_winner_pos_adj[index][-1]
            prev_winner_fan_adj=cum_winner_fan_adj[index][-1]
        cum_winner_pos[index].append(prev_winner_pos+winner_pos[index][file_id])
        cum_winner_fan[index].append(prev_winner_fan+winner_fan[index][file_id])
        cum_winner_pos_adj[index].append(prev_winner_pos_adj+winner_pos_adj[index][file_id])
        cum_winner_fan_adj[index].append(prev_winner_fan_adj+winner_fan_adj[index][file_id])

In [25]:
peek_index=45000
cum = 0
for i in range(4):
    cum+=cum_winner_pos_adj[i][peek_index]
for i in range(4):
    print("Pos {}'s adjusted win rate is {:.3f}".format(i,cum_winner_pos_adj[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_pos_adj[0][peek_index]-cum_winner_pos_adj[3][peek_index])/cum))

Pos 0's adjusted win rate is 0.257
Pos 1's adjusted win rate is 0.253
Pos 2's adjusted win rate is 0.248
Pos 3's adjusted win rate is 0.242
Maximum Diff is 0.015201702239673591


In [26]:
cum=0
for i in range(4):
    cum+=cum_winner_pos[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_pos[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_pos[0][peek_index]-cum_winner_pos[3][peek_index])/cum))

Pos 0's raw win rate is 0.267
Pos 1's raw win rate is 0.258
Pos 2's raw win rate is 0.246
Pos 3's raw win rate is 0.228
Maximum Diff is 0.039665875479276975


In [27]:
cum=0
for i in range(4):
    cum+=cum_winner_fan[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_fan[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_fan[0][peek_index]-cum_winner_fan[3][peek_index])/cum))

Pos 0's raw win rate is 0.271
Pos 1's raw win rate is 0.256
Pos 2's raw win rate is 0.247
Pos 3's raw win rate is 0.227
Maximum Diff is 0.044259101211629404


In [28]:
cum=0
for i in range(4):
    cum+=cum_winner_fan_adj[i][peek_index]
for i in range(4):
    print("Pos {}'s raw win rate is {:.3f}".format(i,cum_winner_fan_adj[i][peek_index]/cum))
print("Maximum Diff is {}".format((cum_winner_fan_adj[0][peek_index]-cum_winner_fan_adj[3][peek_index])/cum))

Pos 0's raw win rate is 0.260
Pos 1's raw win rate is 0.250
Pos 2's raw win rate is 0.249
Pos 3's raw win rate is 0.241
Maximum Diff is 0.019159461305911837


### Conclusion: By Adding Completing Round Mechanism, initial position advantage can be decreased by ~60%.